In [57]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="white")
import os
import plotly.express as px
from plotly.subplots import make_subplots
from plotly import graph_objects as go
import os
import scipy.stats as st


### Utilities

In [37]:
def replace_e2e(row):
    if(row=="e2e_image"):
        return "VBN"
    elif(row=="e2e"):
        return "LBC"
    else:
        return row.upper()
def replace_obs(row):
    if(row==0):
        return "No" 
    else: 
        return "Yes"
def summarize_log_file(test_df):
    test_df['mean_et'] = test_df['execution_time'].mean()
    test_df['avg_iterations'] = test_df['num_iterations'].mean()
    test_df['moet'] = test_df['execution_time'].max()
    test_df['ml_usage'] = (test_df['using_safety_controller'].value_counts(normalize=True).sort_values()[0])*100
    test_df['missed_deadline_percent'] = (test_df['missed_deadline'].sum()/test_df.shape[0])*100
    return test_df[['ml_controller','speed','runtime']+list(test_df.columns[9:])].drop_duplicates()

def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h

# Analysis of Simulation Benchmarking Experiments

The benchmarking experiments were run on  a Dell XPS-15. The metrics reported in the paper come from these experiments.

In [38]:
cols={0:"time_string",1:"time_taken_lec",2:"time_taken_safety_controller",
      3:"total_time",4:"moet",5:"mean_et",6:"avg_iterations",7:"obstacle_presence"}
algorithms = ["e2e_image_rtreach","e2e_rtreach","sac_rtreach","ars_rtreach"]
speeds = ["0.5","1.0","1.5"]
runtimes = ['10','25']
file_str = "new_emsoft_experiments/benchmark_experiments_{}_track_porto_{}_{}.csv"
dfs = []
count = 1
for alg in algorithms:
    for speed in speeds:
        for rt in runtimes:
            file_path = file_str.format(alg,speed,rt)
            df = pd.read_csv(file_path,header=None).rename(columns=cols)
            df["ml_usage"] = df["time_taken_lec"]/df["total_time"]
            df=df.drop(columns=["time_taken_lec","total_time","time_taken_safety_controller"])
            df["sc_usage"] = 1 - df["ml_usage"]
            df["ml_usage"]=df["ml_usage"]*100
            df["sc_usage"]=df["sc_usage"]*100
            df["ml_controller"] = alg.replace("_rtreach","")
            df["speed"] = speed
            df['runtime'] = rt
            dfs.append(df)

In [39]:
all_dfs = pd.concat(dfs)
all_dfs["ml_controller"]=all_dfs["ml_controller"].apply(replace_e2e)
all_dfs["obstacle_presence"] = all_dfs["obstacle_presence"].apply(replace_obs)
all_dfs['moet'] = all_dfs['moet'] *1000
all_dfs['mean_et'] = all_dfs['mean_et'] * 1000
all_dfs = all_dfs.reset_index().drop(columns='index')
all_dfs.groupby(["ml_controller","speed","runtime"]).count()

time_string  moet  mean_et  avg_iterations  \
ml_controller speed runtime                                               
ARS           0.5   10                58    58       58              58   
                    25                58    58       58              58   
              1.0   10                58    58       58              58   
                    25                58    58       58              58   
              1.5   10                58    58       58              58   
                    25                58    58       58              58   
LBC           0.5   10                59    59       59              59   
                    25                60    60       60              60   
              1.0   10                59    59       59              59   
                    25                58    58       58              58   
              1.5   10                59    59       59              59   
                    25                58    58       58              58   
SAC           0.5   10                58    58       58              58   
                    25                58    58       58              58   
              1.0   10                59    59       59              59   
                    25                59    59       59              59   
              1.5   10                58    58       58              58   
                    25                58    58       58              58   
VBN           0.5   10                59    59       59              59   
                    25                60    60       60              60   
              1.0   10                59    59       59              59   
                    25                58    58       58              58   
              1.5   10                59    59       59              59   
                    25                60    60       60              60   

                             obstacle_presence  ml_usage  sc_usage  
ml_controller speed runtime                                         
ARS           0.5   10                      58        58        58  
                    25                      58        58        58  
              1.0   10                      58        58        58  
                    25                      58        58        58  
              1.5   10                      58        58        58  
                    25                      58        58        58  
LBC           0.5   10                      59        58        58  
                    25                      60        59        59  
              1.0   10                      59        59        59  
                    25                      58        58        58  
              1.5   10                      59        59        59  
                    25                      58        58        58  
SAC           0.5   10                      58        58        58  
                    25                      58        58        58  
              1.0   10                      59        59        59  
                    25                      59        59        59  
              1.5   10                      58        58        58  
                    25                      58        58        58  
VBN           0.5   10                      59        59        59  
                    25                      60        60        60  
              1.0   10                      59        59        59  
                    25                      58        58        58  
              1.5   10                      59        59        59  
                    25                      60        60        60

In [40]:
res = round(all_dfs.groupby(["ml_controller","speed","obstacle_presence","runtime"]).agg(["mean","std"]),3)
res

moet         mean_et         \
                                                 mean     std    mean    std   
ml_controller speed obstacle_presence runtime                                  
ARS           0.5   No                10       17.237  10.480   6.353  0.122   
                                      25       30.333   8.790  14.515  0.409   
                    Yes               10       12.980   4.884   6.416  0.127   
                                      25       29.470   5.175  14.875  0.392   
              1.0   No                10       20.067  12.630   6.347  0.156   
                                      25       34.269  10.094  14.852  0.466   
                    Yes               10       15.426   6.428   6.279  0.654   
                                      25       34.534  17.711  15.094  0.556   
              1.5   No                10       12.155   3.536   6.554  0.217   
                                      25       27.831   0.633  15.118  0.381   
                    Yes               10       13.172   6.994   6.165  1.022   
                                      25       30.315   8.309  15.198  0.714   
LBC           0.5   No                10       10.436   2.031   6.648  1.237   
                                      25       27.401   5.163  14.640  2.726   
                    Yes               10       11.730   0.983   6.660  0.996   
                                      25       29.326   1.718  15.203  0.905   
              1.0   No                10       10.678   0.791   6.904  0.088   
                                      25       28.687   0.732  15.154  0.260   
                    Yes               10       12.209   3.691   6.803  0.113   
                                      25       32.263  11.404  15.201  1.578   
              1.5   No                10       15.028  20.747   6.924  0.175   
                                      25       28.460   2.454  15.342  0.761   
                    Yes               10       12.571   2.271   6.801  0.150   
                                      25       28.942   0.930  15.717  1.282   
SAC           0.5   No                10       11.976   3.163   6.437  0.118   
                                      25       28.411   2.721  15.168  0.514   
                    Yes               10       11.439   1.001   6.433  0.137   
                                      25       42.939  73.999  15.359  0.582   
              1.0   No                10       11.895   3.361   6.629  0.284   
                                      25       29.030   6.273  14.491  0.393   
                    Yes               10       11.763   2.552   6.524  0.319   
                                      25       31.586  15.927  14.671  0.470   
              1.5   No                10       11.972   1.886   6.418  0.307   
                                      25       28.383   2.627  14.168  0.499   
                    Yes               10       13.869   5.009   6.429  0.450   
                                      25       29.657   5.570  14.624  0.763   
VBN           0.5   No                10       10.713   0.971   7.005  0.134   
                                      25       28.488   5.228  15.416  0.881   
                    Yes               10       11.658   1.400   6.846  0.136   
                                      25       27.810   1.223  15.129  1.838   
              1.0   No                10       11.099   1.254   6.850  0.158   
                                      25       28.359   0.770  14.865  0.141   
                    Yes               10       14.306  10.355   6.556  1.004   
                                      25       28.864   1.763  14.895  2.029   
              1.5   No                10       11.361   0.597   6.830  0.156   
                                      25       28.188   0.529  15.002  0.252   
                    Yes               10       11.493   0.882   6.741  0.663   
                                      25       28

# Controller Use Analysis

In [41]:
latex=res.reset_index()
table1 = latex[latex.obstacle_presence=="Yes"]
table1 = round(table1.groupby(["ml_controller","speed"]).mean(),2)
print(table1[table1.columns[6:8]].to_latex())

\begin{tabular}{llrr}
\toprule
    &     & \multicolumn{2}{l}{ml\_usage} \\
    &     &     mean &    std \\
ml\_controller & speed &          &        \\
\midrule
ARS & 0.5 &    66.62 &   9.72 \\
    & 1.0 &    46.18 &   7.57 \\
    & 1.5 &    26.50 &   7.56 \\
LBC & 0.5 &    83.23 &  14.25 \\
    & 1.0 &    61.40 &  21.96 \\
    & 1.5 &    32.74 &  12.36 \\
SAC & 0.5 &    42.59 &  17.80 \\
    & 1.0 &    13.20 &   8.42 \\
    & 1.5 &     9.47 &   4.19 \\
VBN & 0.5 &    78.26 &  12.74 \\
    & 1.0 &    53.98 &  15.51 \\
    & 1.5 &    37.97 &  14.11 \\
\bottomrule
\end{tabular}



/home/musaup/anaconda3/envs/d4sa/lib/python3.6/site-packages/pandas/core/generic.py:3889: PerformanceWarning:

dropping on a non-lexsorted multi-index without a level parameter may impact performance.



In [42]:
latex=res.reset_index()
table1 = latex[latex.obstacle_presence=="No"]
table1 = round(table1.groupby(["ml_controller","speed"]).mean(),2)
print(table1[table1.columns[6:8]].to_latex())

\begin{tabular}{llrr}
\toprule
    &     & \multicolumn{2}{l}{ml\_usage} \\
    &     &     mean &    std \\
ml\_controller & speed &          &        \\
\midrule
ARS & 0.5 &    69.01 &   8.98 \\
    & 1.0 &    49.99 &   3.85 \\
    & 1.5 &    30.79 &   5.02 \\
LBC & 0.5 &    99.32 &   1.07 \\
    & 1.0 &    94.83 &   6.95 \\
    & 1.5 &    43.67 &  16.92 \\
SAC & 0.5 &    50.32 &  10.29 \\
    & 1.0 &    11.13 &   8.36 \\
    & 1.5 &     8.87 &   3.94 \\
VBN & 0.5 &    94.08 &   2.92 \\
    & 1.0 &    78.56 &   4.34 \\
    & 1.5 &    53.07 &   9.89 \\
\bottomrule
\end{tabular}



## Analysis from log files without Obstacles

In [43]:
cols={0:"iteration",1:"using_safety_controller",2:"execution_time",
      3:"num_iterations",4:"obstacles"}
algorithms = ["e2e_image_rtreach","e2e_rtreach","sac_rtreach","ars_rtreach"]
speeds = ["0.5","1.0","1.5"]
runtimes = ['10','25']
file_str = "new_emsoft_experiments/logs/benchmark_experiments_{}_track_porto_{}_{}_{}.csv"
dfs = []
count = 1
for alg in algorithms:
    for speed in speeds:
        for rt in runtimes:
            for i in range(1,31):
                file_path = file_str.format(alg,speed,rt,i)
                try:
                    df = pd.read_csv(file_path,header=None).rename(columns=cols)
                    df  = df[df.obstacles==0]
                    df['ml_controller'] = alg.replace("_rtreach","")
                    df['speed'] = speed
                    df['runtime'] = rt
                    df['execution_time'] = df['execution_time'] *1000
                    df['missed_deadline'] = df['execution_time'].astype(float) > df['runtime'].astype(float)
                    df['missed_deadline'] = df['missed_deadline'].astype(int)
                    new_df = summarize_log_file(df)
                    dfs.append(new_df)
                except:
                    print(file_path)
                

In [44]:
new_exps = pd.concat(dfs)
col_order = ['ml_controller', 'speed', 'runtime','moet','mean_et','avg_iterations', 'ml_usage', 'missed_deadline_percent']
new_exps["ml_controller"]=new_exps["ml_controller"].apply(replace_e2e)
new_exps = new_exps[col_order]
new_exps = round(new_exps.groupby(['ml_controller','speed','runtime']).agg(['mean','std']),2)
new_exps

moet        mean_et       avg_iterations        \
                              mean    std    mean   std           mean   std   
ml_controller speed runtime                                                    
ARS           0.5   10       17.24  10.48    6.35  0.12           4.69  0.04   
                    25       30.33   8.79   14.51  0.41           5.79  0.08   
              1.0   10       20.07  12.63    6.35  0.16           4.91  0.12   
                    25       34.27  10.09   14.85  0.47           5.86  0.14   
              1.5   10       12.15   3.54    6.55  0.22           4.08  0.19   
                    25       27.83   0.63   15.12  0.38           5.15  0.08   
LBC           0.5   10       10.78   0.62    6.87  0.10           3.93  0.02   
                    25       28.31   0.91   15.13  0.22           5.03  0.03   
              1.0   10       10.68   0.79    6.90  0.09           3.91  0.02   
                    25       28.69   0.73   15.15  0.26           5.06  0.08   
              1.5   10       15.03  20.75    6.92  0.18           3.97  0.29   
                    25       28.46   2.45   15.34  0.76           5.08  0.19   
SAC           0.5   10       11.98   3.16    6.44  0.12           4.11  0.24   
                    25       28.41   2.72   15.17  0.51           5.31  0.32   
              1.0   10       11.89   3.36    6.63  0.28           4.55  0.47   
                    25       29.03   6.27   14.49  0.39           5.57  0.55   
              1.5   10       11.97   1.89    6.42  0.31           4.92  0.63   
                    25       28.38   2.63   14.17  0.50           6.12  0.62   
VBN           0.5   10       10.71   0.97    7.00  0.13           3.95  0.03   
                    25       28.49   5.23   15.42  0.88           5.07  0.12   
              1.0   10       11.10   1.25    6.85  0.16           4.08  0.23   
                    25       28.36   0.77   14.86  0.14           5.06  0.10   
              1.5   10       11.36   0.60    6.83  0.16           4.04  0.14   
                    25       28.19   0.53   15.00  0.25           5.06  0.03   

                            ml_usage        missed_deadline_percent        
                                mean    std                    mean   std  
ml_controller speed runtime                                                
ARS           0.5   10         69.16   8.29                    0.35  0.18  
                    25         68.85   9.67                    4.62  1.47  
              1.0   10         49.55   4.05                    0.96  0.33  
                    25         50.43   3.65                    4.71  1.34  
              1.5   10         29.78   5.56                    0.53  0.35  
                    25         31.81   4.48                    4.53  1.61  
LBC           0.5   10         98.64   2.14                    0.12  0.09  
                    25        100.00   0.00                    2.40  1.37  
              1.0   10         94.55   6.40                    0.12  0.10  
                    25         95.10   7.49                    3.56  1.08  
              1.5   10         44.27  16.79                    0.27  0.22  
                    25         43.07  17.04                    1.87  1.53  
SAC           0.5   10         50.66   9.99                    0.36  0.30  
                    25         49.98  10.58                    4.73  1.30  
              1.0   10         12.16   8.13                    0.43  0.34  
                    25         10.11   8.59                    3.20  1.96  
              1.5   10          8.31   4.48                    0.66  0.43  
                    25          9.42   3.40                    3.55  1.64  
VBN           0.5   10         94.52   2.18                    0.13  0.15  
                    25         93.64   3.67                    2.62  2.71  
              1.0   10         77.15   6.40                    0.27  0.24  
                    25         79.97   2.27  

In [45]:
print(new_exps.to_latex())

\begin{tabular}{lllrrrrrrrrrr}
\toprule
    &     &    & \multicolumn{2}{l}{moet} & \multicolumn{2}{l}{mean\_et} & \multicolumn{2}{l}{avg\_iterations} & \multicolumn{2}{l}{ml\_usage} & \multicolumn{2}{l}{missed\_deadline\_percent} \\
    &     &    &   mean &    std &    mean &   std &           mean &   std &     mean &    std &                    mean &   std \\
ml\_controller & speed & runtime &        &        &         &       &                &       &          &        &                         &       \\
\midrule
ARS & 0.5 & 10 &  17.24 &  10.48 &    6.35 &  0.12 &           4.69 &  0.04 &    69.16 &   8.29 &                    0.35 &  0.18 \\
    &     & 25 &  30.33 &   8.79 &   14.51 &  0.41 &           5.79 &  0.08 &    68.85 &   9.67 &                    4.62 &  1.47 \\
    & 1.0 & 10 &  20.07 &  12.63 &    6.35 &  0.16 &           4.91 &  0.12 &    49.55 &   4.05 &                    0.96 &  0.33 \\
    &     & 25 &  34.27 &  10.09 &   14.85 &  0.47 &           5.86 &  0.1

## Collisions

In [46]:
collision_files = os.listdir("new_emsoft_experiments/collsions/")
coll_dfs = []
coll_cols={0:"experiment_number",1:"number_of_cones",2:"object1",
      3:"object2"}
for col_file in collision_files:
    collision_info = col_file.split('_')
    df = pd.read_csv('new_emsoft_experiments/collsions/'+col_file,header=None).rename(columns=coll_cols).drop_duplicates()
    df['speed'] = collision_info[-3]
    df['runtime']  = collision_info[-2]
    if(len(collision_info)==10):
        df['controller'] =  collision_info[2]+'_'+collision_info[3]
    else:
        df['controller'] = collision_info[2]
    coll_dfs.append(df)
coll_dfs=pd.concat(coll_dfs)
coll_res = coll_dfs.groupby(['speed','controller']).count().reset_index().rename(columns={"experiment_number":"Count"})
coll_res[coll_res.columns[:3]]

,speed,controller,Count
0,0.5,ars,6
1,0.5,e2e,4
2,0.5,e2e_image,1
3,0.5,sac,6
4,1.0,ars,4
5,1.0,e2e,13
6,1.0,e2e_image,8
7,1.0,sac,2
8,1.5,ars,6
9,1.5,e2e,34


In [47]:
coll_res = coll_dfs.groupby(['speed']).count()
coll_res

,experiment_number,number_of_cones,object1,object2,4,runtime,controller
speed,,,,,,,
0.5,17,17,17,17,17,17,17
1.0,27,27,27,27,27,27,27
1.5,61,61,61,61,61,61,61


### Since we conducted 3228 experiments compute how many would occur at 1440 by taking the Average

In [48]:
total_collisions = 105+103
total_experiments = 1406+1822
coll_p = total_collisions/total_experiments
colls_at_1440 = coll_p*1440
colls_at_1440

92.78810408921932

# Analysis of Hardware Experiments

In [58]:
cols={0:"iteration",1:"using_safety_controller",2:"execution_time",
      3:"num_iterations",4:"obstacles"}
algorithms = ["vbn","lbc","sac","ars"]
speeds = ["0.5","1.0"]
runtimes = ['10','25']
file_str = "{}_{}_{}"
dfs = []
for alg in algorithms:
    for speed in speeds:
        for rt in runtimes:
            file_path = os.path.join("hardware_experiments/",file_str.format(alg,speed,rt))
            log_files = collision_files = os.listdir(file_path)
            for lg in log_files:
                if 'hardware' in lg:
                    continue
                csv_path = os.path.join(file_path,lg)
                try:
                    df = pd.read_csv(csv_path,header=None).rename(columns=cols)
                    df['ml_controller'] = alg.upper()
                    df['speed'] = speed
                    df['runtime'] = rt
                    df['execution_time'] = df['execution_time'] *1000
                    df['missed_deadline'] = df['execution_time'].astype(float) > df['runtime'].astype(float)
                    df['missed_deadline'] = df['missed_deadline'].astype(int)
                    new_df = summarize_log_file(df)
                    dfs.append(new_df)
                except:
                    print(csv_path)


In [99]:
hw_exps = pd.concat(dfs).sort_values(by='missed_deadline_percent').groupby(['ml_controller','speed','runtime']).head(5)
col_order = ['ml_controller', 'speed', 'runtime','moet','mean_et','avg_iterations', 'ml_usage', 'missed_deadline_percent']
hw_exps = hw_exps[col_order]

In [100]:
#a = hw_exps['moet']
def confidence_interval_low(a):
    val = st.t.interval(0.95, len(a)-1, loc=np.mean(a), scale=st.sem(a))
    return val[0]

def confidence_interval_high(a):
    val = st.t.interval(0.95, len(a)-1, loc=np.mean(a), scale=st.sem(a))
    return val[1]

In [101]:
hw_exps = round(hw_exps.groupby(['ml_controller','speed','runtime']).agg(['mean','std',confidence_interval_low,confidence_interval_high]),2)
hw_exps

/home/musaup/anaconda3/envs/d4sa/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:2023: RuntimeWarning:

invalid value encountered in multiply

/home/musaup/anaconda3/envs/d4sa/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:2024: RuntimeWarning:

invalid value encountered in multiply



moet                                 \
                              mean    std confidence_interval_low   
ml_controller speed runtime                                         
ARS           0.5   10       13.92   6.32                    6.08   
                    25       30.83  15.55                   11.53   
              1.0   10       11.52   1.22                   10.01   
                    25       30.58   9.60                   18.66   
LBC           0.5   10       24.49  17.76                    2.44   
                    25       31.31  14.46                   13.36   
              1.0   10       14.25   1.81                   12.00   
                    25       24.43   0.59                   23.71   
SAC           0.5   10       10.98   0.33                   10.58   
                    25       26.78   2.57                   23.58   
              1.0   10       11.46   0.47                   10.87   
                    25       27.16   6.38                   19.24   
VBN           0.5   10       10.87   0.36                   10.42   
                    25       24.42   0.49                   23.81   
              1.0   10       14.36   5.74                    7.23   
                    25       31.60  17.81                    9.49   

                                                     mean_et        \
                            confidence_interval_high    mean   std   
ml_controller speed runtime                                          
ARS           0.5   10                         21.77    4.94  0.30   
                    25                         50.14    9.55  0.53   
              1.0   10                         13.04    5.32  0.20   
                    25                         42.50    8.89  0.72   
LBC           0.5   10                         46.53    5.35  0.45   
                    25                         49.26    9.65  0.51   
              1.0   10                         16.50    5.18  0.36   
                    25                         25.16    9.35  0.74   
SAC           0.5   10                         11.39    5.10  0.16   
                    25                         29.98   10.91  0.78   
              1.0   10                         12.04    4.73  0.54   
                    25                         35.08    9.69  1.17   
VBN           0.5   10                         11.31    5.07  0.53   
                    25                         25.03   10.41  0.63   
              1.0   10                         21.50    5.56  0.58   
                    25                         53.71    9.36  0.79   

                                                                              \
                            confidence_interval_low confidence_interval_high   
ml_controller speed runtime                                                    
ARS           0.5   10                         4.56                     5.31   
                    25                         8.90                    10.21   
              1.0   10                         5.07                     5.57   
                    25                         7.99                     9.78   
LBC           0.5   10                         4.80                     5.91   
                    25                         9.01                    10.29   
              1.0   10                         4.73                     5.63   
                    25                         8.43                    10.27   
SAC           0.5   10                         4.90                     5.29   
                    25                         9.94                    11.88   
              1.0   10                         4.05                     5.41   
                    25                         8.23                    11.14   
VBN           0.5   10                         4.41                     5.73   
                    25                         9.63                    11.19   
              1.0   10     

In [71]:
print(hw_exps.to_latex())

\begin{tabular}{lllrrlrrlrrlrrlrrlrrlrrlrrlrrlrrl}
\toprule
    &     &    & \multicolumn{6}{l}{moet} & \multicolumn{6}{l}{mean\_et} & \multicolumn{6}{l}{avg\_iterations} & \multicolumn{6}{l}{ml\_usage} & \multicolumn{6}{l}{missed\_deadline\_percent} \\
    &     &    & \multicolumn{3}{l}{mean} & \multicolumn{3}{l}{std} & \multicolumn{3}{l}{mean} & \multicolumn{3}{l}{std} & \multicolumn{3}{l}{mean} & \multicolumn{3}{l}{std} & \multicolumn{3}{l}{mean} & \multicolumn{3}{l}{std} & \multicolumn{3}{l}{mean} & \multicolumn{3}{l}{std} \\
    &     &    &   mean & std &        func &   mean & std &        func &    mean & std &        func &  mean & std &        func &           mean & std &        func &  mean & std &        func &     mean & std &        func &  mean & std &        func &                    mean & std &        func &  mean & std &        func \\
ml\_controller & speed & runtime &        &     &             &        &     &             &         &     &             &       & 

In [18]:
analysis = round(pd.concat(dfs),2)
analysis[analysis.ml_usage<100]

,ml_controller,speed,runtime,mean_et,avg_iterations,moet,ml_usage,missed_deadline_percent
0,VBN,0.5,10,4.39,5.97,11.04,8.25,1.10
0,VBN,0.5,10,4.95,5.10,10.58,11.03,0.46
0,VBN,0.5,10,5.33,4.91,11.00,16.26,3.45
0,VBN,0.5,10,5.01,6.17,11.38,15.07,0.44
0,VBN,0.5,10,5.11,5.64,10.50,2.94,0.45
...,...,...,...,...,...,...,...,...
0,ARS,1.0,25,8.50,7.90,30.46,22.92,0.09
0,ARS,1.0,25,8.66,7.30,23.51,24.41,0.00
0,ARS,1.0,25,9.02,7.75,46.70,28.50,0.08
0,ARS,1.0,25,9.41,6.87,43.55,21.92,0.33


# Old Experiments

In [19]:
cols={0:"time_string",1:"time_taken_lec",2:"time_taken_safety_controller",
      3:"total_time",4:"moet",5:"mean_et",6:"avg_iterations",7:"obstacle_presence"}
algorithms = ["e2e_image_rtreach","e2e_rtreach","sac_rtreach","ars_rtreach"]
speeds = ["0.5","1.0","1.5"]
runtimes = ['10','25']
file_str = "benchmark_experiments_{}_track_porto_{}_{}.csv"
dfs = []
count = 1
for alg in algorithms:
    for speed in speeds:
        for rt in runtimes:
            file_path = file_str.format(alg,speed,rt)
            df = pd.read_csv(file_path,header=None).rename(columns=cols)
            df["ml_usage"] = df["time_taken_lec"]/df["total_time"]
            df=df.drop(columns=["time_taken_lec","total_time","time_taken_safety_controller"])
            df["sc_usage"] = 1 - df["ml_usage"]
            df["ml_usage"]=df["ml_usage"]*100
            df["sc_usage"]=df["sc_usage"]*100
            df["ml_controller"] = alg.replace("_rtreach","")
            df["speed"] = speed
            df['runtime'] = rt
            dfs.append(df)

In [20]:
all_dfs = pd.concat(dfs)
all_dfs["ml_controller"]=all_dfs["ml_controller"].apply(replace_e2e)
all_dfs["obstacle_presence"] = all_dfs["obstacle_presence"].apply(replace_obs)
all_dfs['moet'] = all_dfs['moet'] *1000
all_dfs['mean_et'] = all_dfs['mean_et'] * 1000
all_dfs = all_dfs.reset_index().drop(columns='index')
all_dfs

,time_string,moet,mean_et,avg_iterations,obstacle_presence,ml_usage,sc_usage,ml_controller,speed,runtime
0,01/04/2021/17:06:07,8.478,6.64321,3.96654,No,94.933493,5.066507,VBN,0.5,10
1,01/04/2021/17:07:36,8.315,6.60067,3.97587,No,88.899489,11.100511,VBN,0.5,10
2,01/04/2021/17:09:06,8.507,6.43541,3.98251,No,94.946925,5.053075,VBN,0.5,10
3,01/04/2021/17:10:21,8.885,6.63455,3.96435,No,94.508173,5.491827,VBN,0.5,10
4,01/04/2021/17:11:51,8.353,6.52825,3.98548,No,93.997684,6.002316,VBN,0.5,10
...,...,...,...,...,...,...,...,...,...,...
1817,04/04/2021/17:16:40,26.588,14.33750,5.40917,Yes,33.543118,66.456882,ARS,1.5,25
1818,04/04/2021/17:17:54,31.616,13.74030,6.50090,Yes,20.577657,79.422343,ARS,1.5,25
1819,04/04/2021/17:19:23,27.052,14.65300,5.88859,Yes,27.717371,72.282629,ARS,1.5,25
1820,04/04/2021/17:20:37,29.132,14.57170,5.66425,Yes,25.339281,74.660719,ARS,1.5,25


### Experiments where RL errors out

In [21]:
all_dfs.iloc[np.where(all_dfs['moet']>(3*all_dfs['mean_et']))].groupby(["ml_controller","speed","runtime"]).count().sort_values(by="avg_iterations",ascending=False)

time_string  moet  mean_et  avg_iterations  \
ml_controller speed runtime                                               
ARS           1.0   10                16    16       16              16   
                    25                14    14       14              14   
              0.5   10                12    12       12              12   
                    25                11    11       11              11   
SAC           1.5   10                 8     8        8               8   
ARS           1.5   10                 5     5        5               5   
LBC           1.5   10                 4     4        4               4   
SAC           0.5   10                 4     4        4               4   
              1.0   10                 4     4        4               4   
LBC           1.0   25                 2     2        2               2   
              1.5   25                 2     2        2               2   
SAC           1.0   25                 2     2        2               2   
              1.5   25                 2     2        2               2   
VBN           1.5   10                 2     2        2               2   
ARS           1.5   25                 1     1        1               1   
LBC           0.5   10                 1     1        1               1   

                             obstacle_presence  ml_usage  sc_usage  
ml_controller speed runtime                                         
ARS           1.0   10                      16        16        16  
                    25                      14        14        14  
              0.5   10                      12        12        12  
                    25                      11        11        11  
SAC           1.5   10                       8         8         8  
ARS           1.5   10                       5         5         5  
LBC           1.5   10                       4         4         4  
SAC           0.5   10                       4         4         4  
              1.0   10                       4         4         4  
LBC           1.0   25                       2         2         2  
              1.5   25                       2         2         2  
SAC           1.0   25                       2         2         2  
              1.5   25                       2         2         2  
VBN           1.5   10                       2         2         2  
ARS           1.5   25                       1         1         1  
LBC           0.5   10                       1         1         1

In [22]:
all_dfs.iloc[np.where(all_dfs['moet']>(3*all_dfs['mean_et']))].groupby(["ml_controller","speed","runtime"]).mean()
all_dfs = all_dfs.drop(all_dfs.iloc[np.where(all_dfs['moet']>(3*all_dfs['mean_et']))].index)

In [23]:
res = round(all_dfs.groupby(["ml_controller","speed","obstacle_presence","runtime"]).agg(["mean","std"]),3)
res

moet        mean_et         \
                                                 mean    std    mean    std   
ml_controller speed obstacle_presence runtime                                 
ARS           0.5   No                10       11.004  2.911   5.985  0.257   
                                      25       27.238  1.526  14.075  0.866   
                    Yes               10       10.140  1.999   6.042  0.262   
                                      25       27.566  2.205  14.268  0.773   
              1.0   No                10       14.208  2.390   5.965  0.227   
                                      25       28.380  2.329  14.073  0.512   
                    Yes               10       11.444  2.585   5.920  0.507   
                                      25       28.167  2.870  14.337  0.806   
              1.5   No                10        9.134  1.423   6.273  0.240   
                                      25       27.116  1.718  14.238  0.333   
                    Yes               10       10.463  2.549   6.184  0.267   
                                      25       27.668  1.678  14.327  0.619   
LBC           0.5   No                10        9.027  0.395   6.598  0.092   
                                      25       28.247  0.921  14.498  0.106   
                    Yes               10        9.247  0.497   6.415  0.219   
                                      25       28.719  2.664  14.690  0.212   
              1.0   No                10        9.097  0.494   6.615  0.062   
                                      25       27.852  0.603  14.571  0.201   
                    Yes               10        9.868  1.977   6.350  0.256   
                                      25       28.563  2.132  14.935  0.652   
              1.5   No                10        9.673  1.195   6.675  0.506   
                                      25       27.795  0.755  15.014  1.180   
                    Yes               10        9.837  1.555   6.429  0.225   
                                      25       28.994  2.530  14.919  0.527   
SAC           0.5   No                10        9.816  2.526   6.368  0.186   
                                      25       27.375  1.159  13.986  0.309   
                    Yes               10        8.945  0.723   6.286  0.165   
                                      25       27.660  2.167  14.164  0.469   
              1.0   No                10        9.698  1.746   6.417  0.242   
                                      25       27.390  1.994  13.760  0.502   
                    Yes               10       10.222  1.685   6.144  0.268   
                                      25       27.582  2.431  13.917  1.005   
              1.5   No                10        9.377  1.082   6.294  0.282   
                                      25       27.977  2.111  13.691  0.399   
                    Yes               10       10.090  2.025   6.241  0.320   
                                      25       27.642  1.674  14.030  0.442   
VBN           0.5   No                10        8.497  0.279   6.540  0.176   
                                      25       27.175  0.791  14.444  0.697   
                    Yes               10        8.822  0.703   6.389  0.205   
                                      25       27.496  0.902  14.455  0.286   
              1.0   No                10        8.724  0.377   6.588  0.148   
                                      25       27.392  0.699  14.211  0.221   
                    Yes               10        9.243  1.247   6.309  0.584   
                                      25       27.802  1.884  14.510  0.899   
              1.5   No                10        8.932  0.465   6.554  0.168   
                                      25       27.318  0.937  14.362  0.300   
                    Yes               10        9.219  0.592   6.279  0.562   
                                      25       27.360  0.939  14.603  0.780   

                   

# Generate Latex Tables

In [24]:
latex=res.reset_index()
#latex[latex.obstacle_presence=="Yes"].groupby(["ml_controller","speed","runtime"]).mean()
table1 = latex[latex.obstacle_presence=="Yes"]
#table1 = table1[table1.runtime=='25'].groupby(["ml_controller","runtime"]).mean()
table1 = round(table1.groupby(["ml_controller","runtime"]).mean(),2)
table1
print(table1[table1.columns[2:6]].to_latex())

\begin{tabular}{llrrrr}
\toprule
    &    & \multicolumn{2}{l}{mean\_et} & \multicolumn{2}{l}{avg\_iterations} \\
    &    &    mean &   std &           mean &   std \\
ml\_controller & runtime &         &       &                &       \\
\midrule
ARS & 10 &    6.05 &  0.35 &           4.63 &  0.66 \\
    & 25 &   14.31 &  0.73 &           5.68 &  0.45 \\
LBC & 10 &    6.40 &  0.23 &           4.02 &  0.36 \\
    & 25 &   14.85 &  0.46 &           5.18 &  0.29 \\
SAC & 10 &    6.22 &  0.25 &           4.54 &  0.48 \\
    & 25 &   14.04 &  0.64 &           5.63 &  0.67 \\
VBN & 10 &    6.33 &  0.45 &           4.30 &  1.13 \\
    & 25 &   14.52 &  0.66 &           5.27 &  0.46 \\
\bottomrule
\end{tabular}



/home/musaup/anaconda3/envs/d4sa/lib/python3.6/site-packages/pandas/core/generic.py:3889: PerformanceWarning:

dropping on a non-lexsorted multi-index without a level parameter may impact performance.



In [25]:
latex=res.reset_index()
#latex[latex.obstacle_presence=="Yes"].groupby(["ml_controller","speed","runtime"]).mean()
table1 = latex[latex.obstacle_presence=="No"]
table1 = round(table1.groupby(["ml_controller","runtime"]).mean().reset_index(),2)
print(table1[table1.columns[4:8]].to_latex())

\begin{tabular}{lrrrr}
\toprule
{} & \multicolumn{2}{l}{mean\_et} & \multicolumn{2}{l}{avg\_iterations} \\
{} &    mean &   std &           mean &   std \\
\midrule
0 &    6.07 &  0.24 &           4.62 &  0.14 \\
1 &   14.13 &  0.57 &           5.67 &  0.17 \\
2 &    6.63 &  0.22 &           3.95 &  0.10 \\
3 &   14.69 &  0.50 &           5.11 &  0.19 \\
4 &    6.36 &  0.24 &           4.45 &  0.47 \\
5 &   13.81 &  0.40 &           5.61 &  0.49 \\
6 &    6.56 &  0.16 &           4.01 &  0.14 \\
7 &   14.34 &  0.41 &           5.10 &  0.14 \\
\bottomrule
\end{tabular}



In [26]:
latex=res.reset_index()
#latex[latex.obstacle_presence=="Yes"].groupby(["ml_controller","speed","runtime"]).mean()
table1 = latex[latex.obstacle_presence=="Yes"]
table1 = round(table1.groupby(["ml_controller","speed"]).mean(),2)
print(table1[table1.columns[6:8]].to_latex())

\begin{tabular}{llrr}
\toprule
    &     & \multicolumn{2}{l}{ml\_usage} \\
    &     &     mean &    std \\
ml\_controller & speed &          &        \\
\midrule
ARS & 0.5 &    65.65 &   8.98 \\
    & 1.0 &    42.58 &   8.33 \\
    & 1.5 &    26.62 &   5.91 \\
LBC & 0.5 &    83.48 &  13.13 \\
    & 1.0 &    56.77 &  23.67 \\
    & 1.5 &    34.20 &  12.21 \\
SAC & 0.5 &    48.52 &  18.46 \\
    & 1.0 &    15.39 &   7.94 \\
    & 1.5 &     8.12 &   3.99 \\
VBN & 0.5 &    78.63 &  12.33 \\
    & 1.0 &    55.55 &  16.53 \\
    & 1.5 &    40.19 &  11.94 \\
\bottomrule
\end{tabular}



In [27]:
latex=res.reset_index()
#latex[latex.obstacle_presence=="Yes"].groupby(["ml_controller","speed","runtime"]).mean()
table1 = latex[latex.obstacle_presence=="No"]
table1 = round(table1.groupby(["ml_controller","speed"]).mean().reset_index(),2)
print(table1[table1.columns[8:10]].to_latex())

\begin{tabular}{lrr}
\toprule
{} & \multicolumn{2}{l}{ml\_usage} \\
{} &     mean &    std \\
\midrule
0  &    66.79 &   6.33 \\
1  &    48.14 &   6.72 \\
2  &    32.51 &   5.58 \\
3  &    99.59 &   1.12 \\
4  &    95.90 &   8.66 \\
5  &    37.19 &  17.90 \\
6  &    51.70 &  13.35 \\
7  &    11.33 &   9.08 \\
8  &     8.69 &   4.56 \\
9  &    94.20 &   1.00 \\
10 &    82.10 &   9.91 \\
11 &    55.00 &   8.11 \\
\bottomrule
\end{tabular}



In [28]:
withoutobs = all_dfs[all_dfs.obstacle_presence=="No"]
res = round(withoutobs.groupby(["ml_controller","speed","runtime"]).agg(["mean","std"]),2)
res

moet       mean_et       avg_iterations        \
                              mean   std    mean   std           mean   std   
ml_controller speed runtime                                                   
ARS           0.5   10       11.00  2.91    5.99  0.26           4.77  0.05   
                    25       27.24  1.53   14.08  0.87           5.89  0.14   
              1.0   10       14.21  2.39    5.97  0.23           4.99  0.11   
                    25       28.38  2.33   14.07  0.51           5.95  0.17   
              1.5   10        9.13  1.42    6.27  0.24           4.09  0.26   
                    25       27.12  1.72   14.24  0.33           5.17  0.19   
LBC           0.5   10        9.03  0.39    6.60  0.09           3.93  0.02   
                    25       28.25  0.92   14.50  0.11           5.04  0.02   
              1.0   10        9.10  0.49    6.61  0.06           3.93  0.02   
                    25       27.85  0.60   14.57  0.20           5.04  0.03   
              1.5   10        9.67  1.19    6.68  0.51           4.00  0.25   
                    25       27.80  0.76   15.01  1.18           5.27  0.51   
SAC           0.5   10        9.82  2.53    6.37  0.19           4.18  0.35   
                    25       27.38  1.16   13.99  0.31           5.26  0.32   
              1.0   10        9.70  1.75    6.42  0.24           4.44  0.51   
                    25       27.39  1.99   13.76  0.50           5.69  0.61   
              1.5   10        9.38  1.08    6.29  0.28           4.73  0.54   
                    25       27.98  2.11   13.69  0.40           5.89  0.53   
VBN           0.5   10        8.50  0.28    6.54  0.18           3.98  0.01   
                    25       27.17  0.79   14.44  0.70           5.10  0.08   
              1.0   10        8.72  0.38    6.59  0.15           4.02  0.20   
                    25       27.39  0.70   14.21  0.22           5.08  0.16   
              1.5   10        8.93  0.47    6.55  0.17           4.04  0.22   
                    25       27.32  0.94   14.36  0.30           5.13  0.18   

                            ml_usage        sc_usage         
                                mean    std     mean    std  
ml_controller speed runtime                                  
ARS           0.5   10         67.57   6.42    32.43   6.42  
                    25         66.01   6.24    33.99   6.24  
              1.0   10         47.72   7.29    52.28   7.29  
                    25         48.57   6.14    51.43   6.14  
              1.5   10         30.63   7.69    69.37   7.69  
                    25         34.38   3.47    65.62   3.47  
LBC           0.5   10         99.18   2.24     0.82   2.24  
                    25        100.00   0.00     0.00   0.00  
              1.0   10         96.70   2.74     3.30   2.74  
                    25         95.09  14.59     4.91  14.59  
              1.5   10         35.42  18.69    64.58  18.69  
                    25         38.96  17.11    61.04  17.11  
SAC           0.5   10         52.33  13.43    47.67  13.43  
                    25         51.06  13.28    48.94  13.28  
              1.0   10          9.94   8.13    90.06   8.13  
                    25         12.72  10.02    87.28  10.02  
              1.5   10          8.66   4.74    91.34   4.74  
                    25          8.73   4.38    91.27   4.38  
VBN           0.5   10         94.03   1.62     5.97   1.62  
                    25         94.38   0.37     5.62   0.37  
              1.0   10         80.99  13.44    19.01  13.44  
                    25         83.21   6.39    16.79   6.39  
              1.5   10         55.21   5.89    44.79   5.89  
                    25         54.79  10.33    45.21  10.33

## Collisions 

In [29]:
collision_files = os.listdir("collisions/")
coll_dfs = []
coll_cols={0:"experiment_number",1:"number_of_cones",2:"object1",
      3:"object2"}
for col_file in collision_files:
    collision_info = col_file.split('_')
    df = pd.read_csv('collisions/'+col_file,header=None).rename(columns=coll_cols).drop_duplicates()
    df['speed'] = collision_info[-3]
    df['runtime']  = collision_info[-2]
    if(len(collision_info)==10):
        df['controller'] =  collision_info[2]+'_'+collision_info[3]
    else:
        df['controller'] = collision_info[2]
    coll_dfs.append(df)
coll_dfs=pd.concat(coll_dfs)
coll_res = coll_dfs.groupby(['speed','controller']).count().reset_index().rename(columns={"experiment_number":"Count"})
print(coll_res['Count'].sum())
coll_res[coll_res.columns[:3]]

103


,speed,controller,Count
0,0.5,ars,4
1,0.5,e2e,4
2,0.5,e2e_image,10
3,0.5,sac,3
4,1.0,ars,5
5,1.0,e2e,10
6,1.0,e2e_image,10
7,1.0,sac,1
8,1.5,ars,5
9,1.5,e2e,28


In [30]:
coll_res = coll_dfs.groupby(['speed']).count()
coll_res

,experiment_number,number_of_cones,object1,object2,4,runtime,controller
speed,,,,,,,
0.5,21,21,21,21,21,21,21
1.0,26,26,26,26,26,26,26
1.5,56,56,56,56,56,56,56


# Old Hardware Experiments 

In [31]:
hw_file_str = "benchmark_hardware_experiments_{}_{}_{}.csv"
algorithms = ["e2e_image_rtreach","e2e_rtreach","sac_rtreach","ars_rtreach"]
speeds = ["0.5","1.0"]
runtimes = ['10','25']
hw_dfs = []
for sp in speeds:
    for alg in algorithms:
        for rt in runtimes:
            file_path = hw_file_str.format(sp,alg,rt)
            try:
                df = pd.read_csv(file_path,header=None).rename(columns=cols)
                df["ml_usage"] = df["time_taken_lec"]/df["total_time"]
                df=df.drop(columns=["time_taken_lec","time_taken_safety_controller","total_time","obstacle_presence"])
                df["sc_usage"] = 1 - df["ml_usage"]
                df["ml_usage"]=df["ml_usage"]*100
                df["sc_usage"]=df["sc_usage"]*100
                df['moet'] = df['moet'] * 1000
                df['mean_et'] = df['mean_et'] * 1000
                df["ml_controller"] = alg.replace("_rtreach","")
                df["speed"] = sp
                df['runtime'] = rt
                hw_dfs.append(df)
            except:
                print(file_path)
                continue
hw_dfs = pd.concat(hw_dfs)
hw_dfs["ml_controller"]=hw_dfs["ml_controller"].apply(replace_e2e)
hw_dfs

,time_string,moet,mean_et,avg_iterations,ml_usage,sc_usage,ml_controller,speed,runtime
0,03/04/2021/13:25:54,10.935,5.78091,4.99117,17.917072,82.082928,VBN,0.5,10
1,03/04/2021/13:27:15,11.029,6.30026,4.55825,27.399794,72.600206,VBN,0.5,10
2,03/04/2021/13:28:57,13.587,6.11045,4.89243,26.247901,73.752099,VBN,0.5,10
3,03/04/2021/13:30:38,24.368,5.69501,4.36828,20.250907,79.749093,VBN,0.5,10
4,03/04/2021/13:32:13,12.129,5.33000,4.88028,17.869757,82.130243,VBN,0.5,10
...,...,...,...,...,...,...,...,...,...
0,03/04/2021/13:56:42,24.684,11.01770,5.55328,19.607765,80.392235,ARS,1.0,25
1,03/04/2021/13:58:20,23.122,11.38650,5.33877,15.694671,84.305329,ARS,1.0,25
2,03/04/2021/13:59:56,24.557,11.03970,5.27128,29.077913,70.922087,ARS,1.0,25
3,03/04/2021/14:01:38,25.477,11.30330,5.77496,26.734366,73.265634,ARS,1.0,25


In [32]:
res = round(hw_dfs.groupby(["ml_controller","speed","runtime"]).agg(["mean","std"]),2)
res[res.columns[:-2]]
print(res[res.columns[:-2]].to_latex())

\begin{tabular}{lllrrrrrrrr}
\toprule
    &     &    & \multicolumn{2}{l}{moet} & \multicolumn{2}{l}{mean\_et} & \multicolumn{2}{l}{avg\_iterations} & \multicolumn{2}{l}{ml\_usage} \\
    &     &    &   mean &   std &    mean &   std &           mean &   std &     mean &    std \\
ml\_controller & speed & runtime &        &       &         &       &                &       &          &        \\
\midrule
ARS & 0.5 & 10 &  13.81 &  6.39 &    4.88 &  0.26 &           4.90 &  0.71 &    20.22 &   5.30 \\
    &     & 25 &  23.74 &  0.64 &   11.27 &  0.66 &           5.95 &  0.63 &    33.94 &   8.15 \\
    & 1.0 & 10 &  11.21 &  0.87 &    5.09 &  0.42 &           4.64 &  0.46 &    34.24 &  36.91 \\
    &     & 25 &  24.53 &  0.86 &   11.15 &  0.18 &           5.54 &  0.24 &    22.64 &   5.38 \\
LBC & 0.5 & 10 &  13.36 &  2.61 &    6.22 &  0.52 &           4.77 &  0.16 &    21.75 &   6.90 \\
    &     & 25 &  24.82 &  0.99 &   11.34 &  0.64 &           5.37 &  0.19 &    29.15 &   5.57 \\
    &

# Extra Info

In [33]:
hw_file_str = "benchmark_hardware_experiments_{}_{}_{}.csv"
algorithms = ["e2e_image_rtreach","e2e_rtreach","sac_rtreach","ars_rtreach"]
speeds = ["0.5","1.0"]
runtimes = ['10','25']
hw_dfs = []
for sp in speeds:
    for alg in algorithms:
        for rt in runtimes:
            file_path = hw_file_str.format(sp,alg,rt)
            try:
                df = pd.read_csv(file_path,header=None).rename(columns=cols)
                df["ml_usage"] = df["time_taken_lec"]/df["total_time"]
                df=df.drop(columns=["time_taken_lec","time_taken_safety_controller","obstacle_presence"])
                df["sc_usage"] = 1 - df["ml_usage"]
                df["ml_usage"]=df["ml_usage"]*100
                df["sc_usage"]=df["sc_usage"]*100
                df['moet'] = df['moet'] * 1000
                df['mean_et'] = df['mean_et'] * 1000
                df["ml_controller"] = alg.replace("_rtreach","")
                df["speed"] = sp
                df['runtime'] = rt
                hw_dfs.append(df)
            except:
                print(file_path)
                continue
hw_dfs = pd.concat(hw_dfs)
hw_dfs["ml_controller"]=hw_dfs["ml_controller"].apply(replace_e2e)
hw_dfs['total_time'].describe()

count    83.000000
mean     11.488623
std       3.691032
min       0.208044
25%       8.107125
50%      12.216000
75%      14.487250
max      17.943400
Name: total_time, dtype: float64

In [34]:
cols={0:"time_string",1:"time_taken_lec",2:"time_taken_safety_controller",
      3:"total_time",4:"moet",5:"mean_et",6:"avg_iterations",7:"obstacle_presence"}
algorithms = ["e2e_image_rtreach","e2e_rtreach","sac_rtreach","ars_rtreach"]
speeds = ["0.5","1.0","1.5"]
runtimes = ['10','25']
file_str = "benchmark_experiments_{}_track_porto_{}_{}.csv"
dfs = []
count = 1
for alg in algorithms:
    for speed in speeds:
        for rt in runtimes:
            file_path = file_str.format(alg,speed,rt)
            df = pd.read_csv(file_path,header=None).rename(columns=cols)
            df["ml_usage"] = df["time_taken_lec"]/df["total_time"]
            df=df.drop(columns=["time_taken_lec","time_taken_safety_controller"])
            df["sc_usage"] = 1 - df["ml_usage"]
            df["ml_usage"]=df["ml_usage"]*100
            df["sc_usage"]=df["sc_usage"]*100
            df["ml_controller"] = alg.replace("_rtreach","")
            df["speed"] = speed
            df['runtime'] = rt
            dfs.append(df)
            
all_dfs = pd.concat(dfs)
all_dfs['total_time'].describe()

count    1822.000000
mean       15.526192
std         5.541865
min         0.749479
25%        10.446225
50%        14.731950
75%        18.613550
max        33.846400
Name: total_time, dtype: float64